In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
# 载入数据集
mnist = input_data.read_data_sets(r"C:\Users\zdwxx\Downloads\Compressed\MNIST_data", one_hot=True)

# 定义每个批次的大小
batch_size = 100
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

def varible_summaries(var):
    
    with tf.name_scope("summary"):
        
        mean = tf.reduce_mean(var)
        tf.summary.scalar("mean", mean) # 平均值
        with tf.name_scope("stddev"):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar("stddev", stddev) # 标准差
        tf.summary.scalar("max", tf.reduce_max(var)) #最大值
        tf.summary.scalar("min", tf.reduce_min(var)) # 最小值
        tf.summary.histogram("histogram", var) # 直方图

# 命名空间
with tf.name_scope("input"):
    # 定义两个placeholder
    x = tf.placeholder(tf.float32, [None, 784], name="x-input")
    y = tf.placeholder(tf.float32, [None, 10], name="y-input")

with tf.name_scope("layer"):
    #创建一个简单的神经网络
    with tf.name_scope("wights"):
        W = tf.Variable(tf.zeros([784, 10]), name="W")
        varible_summaries(W)
        
    with tf.name_scope("biases"):
        b = tf.Variable(tf.zeros([10]), name="b")
        varible_summaries(b)
        
    with tf.name_scope("wx_plus_b"):
        wx_plus_b = tf.matmul(x, W) + b
        
    with tf.name_scope("softmax"):
        prediction = tf.nn.softmax(wx_plus_b) # 预测值

# 二次代价函数

# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.square(y - prediction))
    tf.summary.scalar("loss", loss)
    
# 梯度下降法
with tf.name_scope("train"):
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

# 初始化变量
init = tf.global_variables_initializer()

with tf.name_scope("accuracy"):
    # 结果存放在一个布尔型列表中
    with tf.name_scope("correct_prediction"):
        correct_prediction = tf.equal(tf.arg_max(y, 1), tf.arg_max(prediction, 1)) #arg_max返回1维张量中最大的值所在的位置
    # 求准确率
    with tf.name_scope("accuracy"):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))#cast转换类型，True->1.0, False->0.0
        tf.summary.scalar("accuracy", accuracy)

# 合并所有的summary
merged = tf.summary.merge_all()
        
        
with tf.Session() as sess:
    
    run_times = 0
    sess.run(init)
    writer = tf.summary.FileWriter(r"C:\logs", sess.graph)
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size) #类似于read，一次读取100张图片
            summary = sess.run([train_step, merged], feed_dict={x : batch_xs, y : batch_ys})[1]
            writer.add_summary(summary, run_times)
            run_times += 1
        
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print("第", epoch, "个周期", "准确率是", acc)